In [7]:
import geopandas as gpd
import xarray as xr
import json, sys, os, pathlib
import numpy as np
import pandas as pd

## Solve instance
This is the most complex notebook of the workflow. It reads all information related to one instance (depot) and then prepares the problem (in JSON format) for the VROOM solver. The solution is read and aggregated at different levels.

In [9]:
## Manage inputs and outputs

# Depot and parcels
depots_path = "../../results/depots/individual/dhl/13.gpkg"
parcels_path = "../../results/parcels/per_depot/baseline_2022/dhl/13.gpkg"

# Routing input
matrix_paths = {
    "default": "../../results/matrices/baseline_2022/dhl/13/default.nc",
    #"accessible": "../../results/matrices/baseline_2022/dhl/13/accessible.nc"
}

# Scenario configuration
configuration_path = "../../results/instances/baseline_2022/dhl/13.json"

# Path to the VROOM executable
vroom_path = "../../results/vroom/bin/vroom"

# Output to stops, legs, and vehicles
output_stops_path = "../../results/stops.parquet"
output_legs_path = "../../results/legs.parquet"
output_tours_path = "../../results/tours.parquet"
output_vehicles_path = "../../results/vehicles.parquet"
output_instance_path = "../../results/instance.parquet"

# Parameters: operator name, depot identifier
operator = "dhl"
depot = 13

# Path from where to import the library
import_path = (pathlib.Path(os.getcwd()).parent / "src").as_posix()

# Runtime: threads, time limit and working directory
threads = 12
time_limit_s = 7200
working_directory = "/home/shoerl/temp"

if "snakemake" in locals():
    # Depot and parcels
    depots_path = snakemake.input["depots"]
    parcels_path = snakemake.input["parcels"]

    # Routing input
    matrix_paths = {}

    for key in snakemake.input.keys():
        if key.startswith("matrix:"):
            matrix_paths[key.replace("matrix:", "")] = snakemake.input[key]

    # Scenario configuration
    configuration_path = snakemake.input["configuration"]

    # Path to the VROOM executable
    vroom_path = snakemake.input["vroom"]
    
    # Output to stops, legs, and vehicles
    output_stops_path = snakemake.output["stops"]
    output_legs_path = snakemake.output["legs"]
    output_tours_path = snakemake.output["tours"]
    output_vehicles_path = snakemake.output["vehicles"]
    output_instance_path = snakemake.output["instance"]

    params = snakemake.params[0] if len(snakemake.params) == 1 and len(snakemake.params.keys()) == 0 else snakemake.params

    # Parameters: operator name, depot identifier
    operator = params["operator"]
    depot = params["depot"]

    # Path from where to import the library
    import_path = params["import_path"]

    # Runtime: threads, time limit and working directory
    threads = snakemake.threads
    working_directory = "."
    
    if "time_limit_s" in params.keys():
        time_limit_s = params["time_limit_s"]


assert len(matrix_paths) > 0

In [10]:
# Import the library for VROOM
sys.path.insert(0, import_path)
import solver, vroom

### Reading input data

In [11]:
# Read depot and parcels
df_depots = gpd.read_file(depots_path)
df_parcels = gpd.read_file(parcels_path)

assert len(df_depots["depot_id"].unique()) == 1
assert len(df_parcels["depot_id"].unique()) == 1
assert df_depots["depot_id"].unique()[0] == df_parcels["depot_id"].unique()[0]

In [12]:
# Read matrices
matrices = {
    network: xr.open_dataarray(path)
    for network, path in matrix_paths.items()
}

### Configuration

In [13]:
with open(configuration_path) as f:
    configuration = json.load(f)[0]

In [14]:
# Clean up configuration
costs = {
    "electricity_EUR_per_kWh": configuration["costs"]["electricity_EUR_per_kWh"],
    "fuel_EUR_per_L": configuration["costs"]["fuel_EUR_per_L"],
    "included_co2eq_EUR_per_ton": configuration["costs"]["included_co2eq_EUR_per_ton"] if "included_co2eq_EUR_per_ton" in configuration["costs"] else 0.0,
    "co2eq_EUR_per_ton": configuration["costs"]["co2eq_EUR_per_ton"] if "co2eq_EUR_per_ton" in configuration["costs"] else 0.0,
    "driver_salary_EUR_per_month": configuration["costs"]["driver_salary_EUR_per_month"],
    "thermic_vehicles_factor": configuration["costs"]["thermic_vehicles_factor"] if "thermic_vehicles_factor" in configuration["costs"] else 1.0,
    "electric_vehicles_factor": configuration["costs"]["electric_vehicles_factor"] if "electric_vehicles_factor" in configuration["costs"] else 1.0,
}

externalities = {
    "co2eq_g_per_kWh": configuration["externalities"]["co2eq_g_per_kWh"],
    "fuel_energy_kWh_per_L": configuration["externalities"]["fuel_energy_kWh_per_L"]
}

delivery_days_per_month = configuration["delivery_days_per_month"]
active_time_h_per_day = configuration["active_time_h_per_day"]
global_maximum_speed_km_h = configuration["maximum_speed_km_h"]
travel_time_factor = configuration["travel_time_factor"]

In [15]:
# Flatten vehicle type configuration
vehicle_types = {}

for vehicle_type_item in configuration["vehicle_types"]:
    vehicle_type = {}
    vehicle_types[vehicle_type_item["type"]] = vehicle_type

    vehicle_type["network"] = vehicle_type_item["network"]
    vehicle_type["maximum_vehicles"] = vehicle_type_item["maximum_vehicles"] if "maximum_vehicles" in vehicle_type_item else None
    
    # switch to vehicle type level
    options = vehicle_type_item["configuration"]

    consumption = options["consumption"]
    assert "fuel_L_per_100km" in consumption or "electricity_Wh_per_km" in consumption

    vehicle_type["fuel_L_per_100km"] = consumption["fuel_L_per_100km"] if "fuel_L_per_100km" in consumption else 0.0
    vehicle_type["electricity_Wh_per_km"] = consumption["electricity_Wh_per_km"] if "electricity_Wh_per_km" in consumption else 0.0
    
    vehicle_type["thermic_co2eq_g_per_km"] = 0.0

    if "externalities" in options:
        vehicle_externalities = options["externalities"]
        vehicle_type["thermic_co2eq_g_per_km"] = vehicle_externalities["co2eq_g_per_km"] if "co2eq_g_per_km" in vehicle_externalities else 0.0

    vehicle_type["energy_cost_EUR_per_km"] = 0.0
    vehicle_type["energy_cost_EUR_per_km"] += vehicle_type["fuel_L_per_100km"] * 1e-2 * costs["fuel_EUR_per_L"]
    vehicle_type["energy_cost_EUR_per_km"] += vehicle_type["electricity_Wh_per_km"] * 1e-3 * costs["electricity_EUR_per_kWh"]

    vehicle_type["co2eq_g_per_km"] = vehicle_type["thermic_co2eq_g_per_km"]
    vehicle_type["co2eq_g_per_km"] += vehicle_type["electricity_Wh_per_km"] * 1e-3 * externalities["co2eq_g_per_kWh"]

    vehicle_type["externalities_cost_EUR_per_km"] = 0.0
    vehicle_type["externalities_cost_EUR_per_km"] += vehicle_type["co2eq_g_per_km"] * 1e-6 * costs["co2eq_EUR_per_ton"]

    if costs["included_co2eq_EUR_per_ton"] > 0.0:
        vehicle_type["externalities_cost_EUR_per_km"] += vehicle_type["co2eq_g_per_km"] * 1e-6 * costs["included_co2eq_EUR_per_ton"]
        vehicle_type["energy_cost_EUR_per_km"] -= vehicle_type["co2eq_g_per_km"] * 1e-6 * costs["included_co2eq_EUR_per_ton"]

    vehicle_type["distance_cost_EUR_per_km"] = vehicle_type["energy_cost_EUR_per_km"] + vehicle_type["externalities_cost_EUR_per_km"]

    is_electric = "electric" in options and options["electric"]
    cost_factor = costs["electric_vehicles_factor"] if is_electric else costs["thermic_vehicles_factor"]

    vehicle_type["vehicle_cost_EUR_per_day"] = cost_factor * options["cost_EUR_per_month"] / delivery_days_per_month
    vehicle_type["driver_cost_EUR_per_day"] = costs["driver_salary_EUR_per_month"] / delivery_days_per_month

    vehicle_type["unit_cost_EUR_per_day"] = vehicle_type["vehicle_cost_EUR_per_day"]
    vehicle_type["unit_cost_EUR_per_day"] += vehicle_type["driver_cost_EUR_per_day"]

    vehicle_type["energy_Wh_per_km"] = vehicle_type["fuel_L_per_100km"] * 1e-2 * externalities["fuel_energy_kWh_per_L"] * 1e3
    vehicle_type["energy_Wh_per_km"] += vehicle_type["electricity_Wh_per_km"]

    vehicle_type["capacity"] = options["capacity"]

    vehicle_type["maximum_speed_km_h"] = np.min([
            options["maximum_speed_km_h"] if "maximum_speed_km_h" in options else np.inf,
            global_maximum_speed_km_h
        ])
    
    vehicle_type["maximum_travel_time_h"] = options["maximum_travel_time_h"] if "maximum_travel_time_h" in options else np.inf
    vehicle_type["maximum_distance_km"] = options["maximum_distance_km"] if "maximum_distance_km" in options else np.inf
    vehicle_type["maximum_travel_time_per_tour_h"] = options["maximum_travel_time_per_tour_h"] if "maximum_travel_time_per_tour_h" in options else np.inf
    vehicle_type["maximum_distance_per_tour_km"] = options["maximum_distance_per_tour_km"] if "maximum_distance_per_tour_km" in options else np.inf

    if vehicle_type_item["type"] == "cargobike":
        vehicle_type["maximum_travel_time_per_tour_h"] = np.inf
        vehicle_type["maximum_distance_per_tour_km"] = np.inf

### Set up VROOM problem

In [16]:
# Define shipments based on parcels
shipments = [
    solver.Shipment(row["parcel_id"], row["node"]) for index, row in df_parcels.iterrows()
]

In [17]:
# Define vehicle types
problem_vehicle_types = {
    name: solver.VehicleType(
        distance_cost_EUR_per_m = vehicle_type["distance_cost_EUR_per_km"] * 1e-3,
        vehicle_cost_EUR_per_day = vehicle_type["unit_cost_EUR_per_day"],
        capacity = vehicle_type["capacity"],
        maximum_speed_km_h = vehicle_type["maximum_speed_km_h"],
        active_time_h = active_time_h_per_day,
        network = vehicle_type["network"],
        maximum_travel_time_h = vehicle_type["maximum_travel_time_h"],
        maximum_distance_km = vehicle_type["maximum_distance_km"],
        maximum_travel_time_per_tour_h = vehicle_type["maximum_travel_time_per_tour_h"],
        maximum_distance_per_tour_km = vehicle_type["maximum_distance_per_tour_km"],
        maximum_vehicles = vehicle_type["maximum_vehicles"]
    )
    for name, vehicle_type in vehicle_types.items()
}

In [18]:
# Define the problem itself
problem = solver.Problem(
    problem_vehicle_types, matrices, 
    configuration["timing"]["pickup_duration"], configuration["timing"]["delivery_duration"],
    shipments,
    df_depots["node"].unique()[0],
    configuration["travel_time_factor"]
)

### Solve the problem

In [19]:
# Create an instance
instance = vroom.VroomInstance(problem)

In [20]:
# Define the executable
executable = vroom.VroomExecutable(
    executable = vroom_path,
    working_directory = working_directory,
    time_limit_s = time_limit_s,
    threads = threads
)

In [21]:
# Solve the problem
solver = vroom.VroomSolver(instance, executable)
df_stops, instance_information = solver.solve()

Solving instance with {'small_thermic': 40, 'medium_thermic': 40, 'large_thermic': 40, 'small_electric': 40, 'medium_electric': 40, 'large_electric': 40}


[Error] bad optional access


KeyError: 'unassigned'

### Postprocessing

In [ ]:
## Derive stops from the solution

# add tour sequence variable
df_stops["tour_sequence"] = 0

for vehicle_id in df_stops["vehicle_id"].unique():
    f_vehicle = df_stops["vehicle_id"] == vehicle_id

    df_vehicle = df_stops[f_vehicle].copy()
    f_start = (df_vehicle["stop_type"] == "pickup") & (df_vehicle["stop_type"].shift(1) == "delivery")
    
    df_stops.loc[f_vehicle, "tour_sequence"] = np.cumsum(f_start)

In [ ]:
## Derive legs from the solution

distance_matrices = {
    vehicle_type: problem.get_distance_matrix(vehicle_type).values
    for vehicle_type in instance._data["matrices"].keys()
}

cost_matrices = {
    vehicle_type: problem.get_cost_matrix(vehicle_type).values
    for vehicle_type in instance._data["matrices"].keys()
}

df_legs = []

for vehicle_id, df_partial in df_stops.groupby("vehicle_id"):
    vehicle_type = df_partial["vehicle_type"].values[0]

    df_partial = pd.DataFrame({
        "origin_index": df_partial["location_index"].values[:-1],
        "destination_index": df_partial["location_index"].values[1:],
        "departure_time": df_partial["departure_time"].values[:-1],
        "arrival_time": df_partial["arrival_time"].values[1:],
        "travel_time": df_partial["arrival_time"].values[1:] - df_partial["departure_time"].values[:-1],
        "cost_EUR": [
            cost_matrices[vehicle_type][origin_index, destination_index]
            for origin_index, destination_index in zip(
                df_partial["location_index"].values[:-1], df_partial["location_index"].values[1:]
            )
        ],
        "distance_m": [
            distance_matrices[vehicle_type][origin_index, destination_index]
            for origin_index, destination_index in zip(
                df_partial["location_index"].values[:-1], df_partial["location_index"].values[1:]
            )
        ],
        "tour_sequence": df_partial["tour_sequence"].values[:-1]
    })

    df_partial["vehicle_id"] = vehicle_id
    df_partial["vehicle_type"] = vehicle_type

    df_legs.append(df_partial)

df_legs = pd.concat(df_legs)

In [ ]:
## Convert stop and leg locations to nodes

node_sequence = problem.get_node_sequence()

df_stops["node"] = df_stops["location_index"].apply(
    lambda index: node_sequence[index]
)

df_legs["origin_node"] = df_legs["origin_index"].apply(
    lambda index: node_sequence[index]
)

df_legs["destination_node"] = df_legs["destination_index"].apply(
    lambda index: node_sequence[index]
)

df_stops["parcel_id"] = df_stops["shipment_index"].apply(
    lambda index: shipments[index].id
)

In [ ]:
## Verify network constraints
networks_by_vehicle_type = {
    item["type"]: item["network"]
    for item in configuration["vehicle_types"]
}

for (vehicle_id, vehicle_type), df_sequence in df_stops.groupby(["vehicle_id", "vehicle_type"]):
    sequence = df_sequence["node"].values
    assert problem.verify_sequence(networks_by_vehicle_type[vehicle_type], sequence)

### Analysis

In [ ]:
## Analyse legs for costs, externalities, energy, ...

df_legs["fuel_L"] = 0.0
df_legs["electricity_Wh"] = 0.0
df_legs["co2eq_g"] = 0.0
df_legs["energy_Wh"] = 0.0

df_legs["distance_cost_EUR"] = 0.0
df_legs["energy_cost_EUR"] = 0.0
df_legs["externalities_cost_EUR"] = 0.0

for vt, options in vehicle_types.items():
    f = df_legs["vehicle_type"] == vt

    df_legs.loc[f, "fuel_L"] = df_legs.loc[f, 
        "distance_m"] * 1e-3 * 1e-2 * options["fuel_L_per_100km"]
    
    df_legs.loc[f, "electricity_Wh"] = df_legs.loc[f, 
        "distance_m"] * 1e-3 * options["electricity_Wh_per_km"]
    
    df_legs.loc[f, "co2eq_g"] = df_legs.loc[f, 
        "distance_m"] * 1e-3 *  options["co2eq_g_per_km"]
    
    df_legs.loc[f, "energy_Wh"] = df_legs.loc[f, 
        "distance_m"] * 1e-3 *  options["energy_Wh_per_km"]

    df_legs.loc[f, "distance_cost_EUR"] = df_legs.loc[f, 
        "distance_m"] * 1e-3 *  options["distance_cost_EUR_per_km"]
    
    df_legs.loc[f, "energy_cost_EUR"] = df_legs.loc[f, 
        "distance_m"] * 1e-3 *  options["energy_cost_EUR_per_km"]

    df_legs.loc[f, "externalities_cost_EUR"] = df_legs.loc[f, 
        "distance_m"] * 1e-3 *  options["externalities_cost_EUR_per_km"]
    
df_legs["duration_min"] = (df_legs["arrival_time"] - df_legs["departure_time"]) / 60.0

In [ ]:
## Analyse tours

df_tours = df_legs.groupby(["vehicle_id", "tour_sequence"]).aggregate({
    "vehicle_type": "first",
    "duration_min": "sum",
    "distance_m": "sum",
    "fuel_L": "sum",
    "electricity_Wh": "sum",
    "co2eq_g": "sum",
    "energy_Wh": "sum",
    "distance_cost_EUR": "sum",
    "energy_cost_EUR": "sum",
    "externalities_cost_EUR": "sum",
    "departure_time": "first",
    "arrival_time": "last",
    "origin_node": "first",
    "destination_node": "last"
}).reset_index()

df_deliveries = df_stops[
    df_stops["stop_type"] == "delivery"].groupby(["vehicle_id", "tour_sequence"]).size().reset_index(name = "deliveries")

df_tours = pd.merge(df_tours, df_deliveries)

In [ ]:
## Analyse vehicles

df_vehicles = df_tours.groupby("vehicle_id").aggregate({
    "vehicle_type": "first",
    "duration_min": "sum",
    "distance_m": "sum",
    "fuel_L": "sum",
    "electricity_Wh": "sum",
    "co2eq_g": "sum",
    "energy_Wh": "sum",
    "distance_cost_EUR": "sum",
    "energy_cost_EUR": "sum",
    "externalities_cost_EUR": "sum",
    "departure_time": "first",
    "arrival_time": "last",
    "origin_node": "first",
    "destination_node": "last",
    "deliveries": "sum"
}).reset_index()

df_vehicles = df_vehicles.rename(columns = { "cost_EUR": "distance_cost_EUR" })
df_vehicles["vehicle_cost_EUR"] = 0.0
df_vehicles["driver_cost_EUR"] = 0.0
df_vehicles["unit_cost_EUR"] = 0.0

for vt, options in vehicle_types.items():
    f = df_vehicles["vehicle_type"] == vt
    
    df_vehicles.loc[f, "vehicle_cost_EUR"] = options["vehicle_cost_EUR_per_day"]
    df_vehicles.loc[f, "driver_cost_EUR"] = options["driver_cost_EUR_per_day"]
    df_vehicles.loc[f, "unit_cost_EUR"] = options["unit_cost_EUR_per_day"]

df_vehicles["total_cost_EUR"] = df_vehicles["distance_cost_EUR"]
df_vehicles["total_cost_EUR"] += df_vehicles["unit_cost_EUR"]

In [ ]:
## Instance information
df_instance = df_vehicles.copy()
df_instance["unique"] = "unique"

df_instance = df_instance.groupby("unique").aggregate({
    "distance_m": "sum",
    "duration_min": "sum",
    "fuel_L": "sum",
    "electricity_Wh": "sum",
    "co2eq_g": "sum",
    "energy_Wh": "sum",
    "distance_cost_EUR": "sum",
    "energy_cost_EUR": "sum",
    "externalities_cost_EUR": "sum",
    "vehicle_cost_EUR": "sum",
    "driver_cost_EUR": "sum",
    "unit_cost_EUR": "sum",
    "total_cost_EUR": "sum",
    "deliveries": "sum"
}).reset_index().drop(columns = ["unique"])

df_instance["runtime:writing"] = instance_information["computing_times"]["python_writing"]
df_instance["runtime:reading"] = instance_information["computing_times"]["python_reading"]
df_instance["runtime:vroom_loading"] = instance_information["computing_times"]["loading"]
df_instance["runtime:vroom_solving"] = instance_information["computing_times"]["solving"]
df_instance["retries"] = instance_information["retries"]

In [ ]:
## Add operator and depot information for later aggregation

for df in [df_stops, df_legs, df_tours, df_vehicles, df_instance]:
    df["operator"] = operator
    df["depot"] = depot

    if "vehicle_id" in df:
        df["vehicle_id"] = df["operator"].astype(str) + ":" + df["depot"].astype(str) + ":" + df["vehicle_id"].astype(str)

In [ ]:
## Select relevant attributes

df_stops = df_stops[[
    "vehicle_id", "operator", "depot", "vehicle_type",
    "stop_sequence", "stop_type",
    "arrival_time", "departure_time",
    "node", "parcel_id",
    "tour_sequence"
]].copy()

df_legs = df_legs[[
    "vehicle_id", "operator", "depot", "vehicle_type",
    "departure_time", "arrival_time",
    "origin_node", "destination_node",
    "tour_sequence",
    "distance_m", "duration_min",
    "fuel_L", "electricity_Wh", "co2eq_g",
    "energy_Wh", "distance_cost_EUR",
    "energy_cost_EUR", "externalities_cost_EUR"
]].copy()

df_tours = df_tours[[
    "vehicle_id", "operator", "depot", "vehicle_type",
    "tour_sequence",
    "departure_time", "arrival_time",
    "origin_node", "destination_node",
    "distance_m", "duration_min",
    "fuel_L", "electricity_Wh", "co2eq_g",
    "energy_Wh", "distance_cost_EUR",
    "energy_cost_EUR", "externalities_cost_EUR",
    "deliveries"
]].copy()

df_vehicles = df_vehicles[[
    "vehicle_id", "operator", "depot", "vehicle_type", "distance_m", "duration_min",
    "fuel_L", "electricity_Wh", "co2eq_g", "energy_Wh", "distance_cost_EUR",
    "energy_cost_EUR", "externalities_cost_EUR", "vehicle_cost_EUR",
    "driver_cost_EUR", "unit_cost_EUR", "total_cost_EUR",
    "deliveries"
]].copy()

df_instance = df_instance[[
    "operator", "depot", "distance_m", "duration_min",
    "fuel_L", "electricity_Wh", "co2eq_g", "energy_Wh", "distance_cost_EUR",
    "energy_cost_EUR", "externalities_cost_EUR", "vehicle_cost_EUR",
    "driver_cost_EUR", "unit_cost_EUR", "total_cost_EUR",
    "deliveries",
    "runtime:writing", "runtime:reading", "runtime:vroom_loading", "runtime:vroom_solving",
    "retries"
]].copy()

### Write output

In [ ]:
df_stops.to_parquet(output_stops_path)
df_legs.to_parquet(output_legs_path)
df_tours.to_parquet(output_tours_path)
df_vehicles.to_parquet(output_vehicles_path)
df_instance.to_parquet(output_instance_path)